In [1]:
"""
Test Script

"""
import random
import copy
import numpy as np
from vwgym import VacuumWorld, Vectorise, StepWrapper
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from vwgym.old_actor_critic import *
from torch.distributions import Categorical
from vwgym.init_utils import *

if torch.cuda.is_available():
    print('GPU Available:\t', True)
    device = 'cuda'
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

else:
    device = 'cpu'

<class 'vacuumworld.vw.Grid'>: size: 3, agents: 1, dirts: 6, 
location(coordinate=coord(x=1, y=0), agent=agent(name='A-1', colour='white', orientation='east'), dirt=dirt(name='D-1', colour='orange'))
location(coordinate=coord(x=2, y=0), agent=None, dirt=dirt(name='D-3', colour='orange'))
location(coordinate=coord(x=0, y=1), agent=None, dirt=dirt(name='D-4', colour='green'))
location(coordinate=coord(x=1, y=1), agent=None, dirt=dirt(name='D-5', colour='green'))
location(coordinate=coord(x=0, y=2), agent=None, dirt=dirt(name='D-2', colour='orange'))
location(coordinate=coord(x=2, y=2), agent=None, dirt=dirt(name='D-6', colour='green'))
GPU Available:	 True


In [2]:
model_path = 'vwgym/model/actor_9190.pt'
checkpoint = torch.load(model_path)

env, input_shape = make_env(grid_size=3, num_env=1, vectorize=True)
env = env[0]
env.state()

<class 'vacuumworld.vw.Grid'>: size: 3, agents: 1, dirts: 6, 
location(coordinate=coord(x=1, y=0), agent=agent(name='A-1', colour='white', orientation='east'), dirt=dirt(name='D-1', colour='orange'))
location(coordinate=coord(x=2, y=0), agent=None, dirt=dirt(name='D-3', colour='orange'))
location(coordinate=coord(x=0, y=1), agent=None, dirt=dirt(name='D-4', colour='green'))
location(coordinate=coord(x=1, y=1), agent=None, dirt=dirt(name='D-5', colour='green'))
location(coordinate=coord(x=0, y=2), agent=None, dirt=dirt(name='D-2', colour='orange'))
location(coordinate=coord(x=2, y=2), agent=None, dirt=dirt(name='D-6', colour='green'))


<class 'vacuumworld.vw.Grid'>: size: 3, agents: 1, dirts: 6, 
location(coordinate=coord(x=1, y=0), agent=agent(name='A-1', colour='white', orientation='east'), dirt=dirt(name='D-1', colour='orange'))
location(coordinate=coord(x=2, y=0), agent=None, dirt=dirt(name='D-3', colour='orange'))
location(coordinate=coord(x=0, y=1), agent=None, dirt=dirt(name='D-4', colour='green'))
location(coordinate=coord(x=1, y=1), agent=None, dirt=dirt(name='D-5', colour='green'))
location(coordinate=coord(x=0, y=2), agent=None, dirt=dirt(name='D-2', colour='orange'))
location(coordinate=coord(x=2, y=2), agent=None, dirt=dirt(name='D-6', colour='green'))

In [3]:
num_actions = env.action_space.n
ac = torch.load(model_path)

In [4]:
ac

Actor(
  (fc1): Linear(in_features=22, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (actor): Linear(in_features=512, out_features=4, bias=True)
)

In [5]:
def take_action(action_probs):

#     dist = Categorical(action_probs)
    print(action_probs)
#     action = dist.sample()
    action = action_probs.max(1)[1]
    
    return action.cpu().numpy()

In [6]:
with torch.no_grad():
    ac.eval()
    predictions = []

    x = env.reset()
    env.rw_dirts = env.dirts
    print(f'Dirts Present..:{env.rw_dirts}')
        
    x = torch.from_numpy(x).float()
    x = x.view(1, -1).to(device)
    
    step = 0
    prev_action = []
    
    for __ in range(5000):
        action_probs = ac(x)
#         print(action_probs)
        a_t = take_action(action_probs)
        x, reward, done, ep_info = env.step(a_t[0])
        print(x, reward, done)
        
        x = torch.from_numpy(x).float()
        x = x.view(1, -1).to(device)

        predictions.append(a_t[0])
        if done:
            break

Dirts Present..:6
tensor([[9.9368e-01, 6.2423e-03, 7.1674e-05, 6.0812e-06]], device='cuda:0')
[0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.] -1 False
tensor([[1.6380e-02, 2.1610e-02, 9.6163e-01, 3.7911e-04]], device='cuda:0')
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1.] -1 False
tensor([[0.0647, 0.9103, 0.0236, 0.0014]], device='cuda:0')
[1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.] 100 False
tensor([[0.0075, 0.0379, 0.9527, 0.0019]], device='cuda:0')
[0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.] -1 False
tensor([[0.9573, 0.0362, 0.0055, 0.0010]], device='cuda:0')
[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.] -1 False
tensor([[1.3329e-01, 8.6260e-01, 3.6236e-03, 4.8226e-04]], device='cuda:0')
[0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1.] 100 False
tensor([[9.7183e-01, 2.7542e-02, 5.7264e-04, 5.4962e-05]], device='cuda:0')
[0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 

In [7]:
import pandas as pd

In [8]:
p = pd.DataFrame.from_dict({'actions': predictions, 'action_meanings':[env.action_meanings[i] for i in predictions]})
p['action_meanings'].value_counts().reset_index()

,index,action_meanings
0,move,10
1,turn_left,6
2,clean,6
